# DELVE survey progress plots

This notebook generates a set of simple survey status plots.

It was developed using the `ehn37` `conda` environment on the DES cluster at Fermilab.

## Boilerplate

Simple boilerplate with needed `python` imports and notebook configuration.

### Imports

In [ ]:
from IPython.core.display import display, HTML
from functools import partial
from collections import OrderedDict
from io import StringIO
from contextlib import redirect_stdout
import os
import sys
import urllib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import astropy
import astropy.coordinates
import astropy.units as u
import healpy
import requests

These should all be installable with `pip`, and probably `conda` is you prefer.
The versions used in this execution:

In [ ]:
print("python", sys.version)
for p in [np, pd, mpl, cartopy, astropy, healpy, requests]:
    print(p.__name__, p.__version__)

### `jupyter/ipython` magic.

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format = 'svg'

### Configuration

In [ ]:
display(HTML("<style>.container { width:90% !important; }</style>"))
mpl.rcParams['figure.figsize'] = (8, 5)
plt.style.use('ggplot')

### Make the notebook reproducible

In [ ]:
np.random.seed(6563)

## Configuration

Exposures planned for DELVE.

In [ ]:
target_source = 'https://github.com/kadrlica/obztak/blob/delve/obztak/data/delve-target-fields-v10.csv.gz?raw=true'
schedule_source = 'https://github.com/delve-survey/observing/raw/master/data/schedule_through_2019B.txt'

## Constants

In [ ]:
EARTH_DIAMETER = 2*6378140
ORTHO = ccrs.Orthographic(central_longitude=0, central_latitude=-89.99)
LAEA = ccrs.LambertAzimuthalEqualArea(central_longitude=0, central_latitude=-89.99)
STEREO = ccrs.SouthPolarStereo()
PC = ccrs.PlateCarree()
SITE = astropy.coordinates.EarthLocation.of_site('Cerro Tololo')
MJD_EPOCH = pd.to_datetime("1858-11-17T00:00:00+00:00", infer_datetime_format=True)
DECAM_AREA = 2.78 ;# Footprint of DECam good pixels in square degrees

## Get planned DELVE exposures

Actually read the field list from `obztak`:

In [ ]:
all_target_fields = pd.read_csv(target_source, compression='gzip', comment="#")
real_fields_query = "(PRIORITY>0 or PROGRAM=='delve-deep') and not (PROGRAM=='delve-wide' and TILING==4)"
target_fields = all_target_fields.query(real_fields_query)

## Get completed DELVE exposures

There are a few ways of getting the exposure data from the SISPI database. They all ultimately get data from the following query:

In [ ]:
completed_query = """
SELECT id, date, flavor, ra, declination AS decl, filter AS band, exptime,
       qc_fwhm, qc_sky, qc_cloud, qc_teff, qc_eps,
       ha, zd, az, moonangl, lst, propid,
       program, object, seqid,
       hex_id, tiling_id
FROM exposure.exposure
WHERE NOT aborted
  AND ( qc_teff > 0.3 OR (propid IN ('2019A-0305')) OR (object LIKE '%DELVE%') )
  AND exptime >= 30
  AND flavor = 'object'
ORDER BY id
"""

Here, I query the SISPI database using the Fermilab mirror of the DECam telemetry viewer.

In [ ]:
tv_url = 'http://des-ops.fnal.gov:8080/TV/app/Q/index'
post_fields = {'namespace': 'exposure',
               'output': 'csv',
               'sql': completed_query}
result = requests.post(tv_url, post_fields)
decam_completed = pd.read_csv(StringIO(result.content.decode('utf-8')), parse_dates=['date'])

Supplement the queried values.

In [ ]:
decam_completed['mjd'] = (decam_completed['date'] - MJD_EPOCH).dt.total_seconds()/(24*60*60)
decam_completed['good'] = decam_completed.qc_teff > 0.3

Look just at exposure taked as part of DELVE, and extract DELVE hex and tiling ids

In [ ]:
completed = decam_completed.query('propid=="2019A-0305"').copy()
completed['hex'] = completed['object'].str.extract(r'DELVE field: (\d+)-..-.').astype(float)
completed['tiling'] = completed['object'].str.extract(r'DELVE field: \d+-(\d\d)-.').astype(float)

## Map planned exposures

In [ ]:
bands = target_fields['FILTER'].unique()
subplot_size = 4
fig = plt.figure(figsize=(4*subplot_size, len(bands)*subplot_size))

for band_idx, band in enumerate(bands):
    for tiling in np.arange(1,5):
        axes = fig.add_subplot(len(bands), 4, tiling + 4*band_idx, projection=LAEA)
        
        if tiling <= 3:
            these_fields = target_fields.query(f'FILTER=="{band}" and TILING=={tiling}')
            these_completed = completed.query(f'band=="{band}" and tiling=={tiling} and good')
            these_bad = completed.query(f'band=="{band}" and tiling=={tiling} and not good')
            axes.set_title(f'tiling {tiling} in {band}')
        else:
            these_fields = target_fields.query(f'FILTER=="{band}" and TILING>={tiling}')
            these_completed = completed.query(f'band=="{band}" and tiling>={tiling} and good')
            these_bad = completed.query(f'band=="{band}" and tiling>={tiling} and not good')
            axes.set_title(f'tiling >= {tiling} in {band}')
            
        axes.scatter(these_fields.RA, these_fields.DEC, s=1, c='gray', transform=PC)
        axes.scatter(these_bad.ra, these_bad.decl, c='red', s=1, transform=PC)
        axes.scatter(these_completed.ra, these_completed.decl, c='blue', s=1, transform=PC)
        
        axes.set_xlim(-0.8*EARTH_DIAMETER, 0.8*EARTH_DIAMETER)
        axes.set_ylim(-0.8*EARTH_DIAMETER, 0.8*EARTH_DIAMETER)

        axes.set_xlim(reversed(axes.get_xlim()))
        gl = axes.gridlines(crs=PC, draw_labels=False, color='gray')

        meridian_ra = np.arange(0, 360+90, 90)
        meridian_decl = {ra: 30 for ra in meridian_ra}
        meridian_label = {ra: "$%d^\circ$" % ra for ra in meridian_ra}
        meridian_params = {ra: {'horizontalalignment': 'center', 'verticalalignment': 'center', 'color': 'gray', 'weight': 'bold', 'transform': PC} for ra in meridian_ra}
        gl.xlocator = mpl.ticker.FixedLocator(meridian_ra)
        gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
        meridian_labels = [axes.text(ra, meridian_decl[ra], meridian_label[ra], **meridian_params[ra]) for ra in meridian_ra if 1 < ra < 350]

        parallel_decl = np.arange(-90, 60, 30)
        parallel_ra = {decl: 50 for decl in parallel_decl}
        parallel_label = {decl: "$%d^\circ$" % decl for decl in parallel_decl}
        parallel_params = {ra: {'horizontalalignment': 'right', 'verticalalignment': 'center', 'color': 'gray', 'weight': 'bold', 'transform': PC} for ra in parallel_ra}

        gl.ylocator = mpl.ticker.FixedLocator(parallel_decl)
        gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
        parallel_labels = [axes.text(parallel_ra[decl], decl, parallel_label[decl], **parallel_params[decl]) for decl in parallel_decl if 10 > decl > -88]


## Comparison to schedule

In [ ]:
schedule = pd.read_csv(schedule_source, parse_dates=[0], sep="\t").query('propid == "0305"')

In [ ]:
def twilight_times(night_mjds, which_direction='down', which_night='nearest', alt=-14.0, location=SITE, body='sun', tolerance=1e-8, max_iter=5):
    """Find morning or evening twilight using Newton's iterative method
    
    This only works at central latitudes!
    
    Args:
        night_mjds - numpy array of MJDs of nights (integers)
        which_twilight -- 'evening' or 'morning'
        alt -- altitude of twilight, in degrees
        location -- astropy.coordinates.EarthLocation for site
        tolerance -- tolerance for twilight altitude, in degrees
        max_iter -- maximum iterations in Newton's method
        
    Returns:
        numpy array of mjds
    
    """
    event_direction = 1 if which_direction=='down' else -1
    
    night_wraps = {'previous': 0.0, 'nearest': 180.0, 'next': 360.0}
    night_wrap = night_wraps[which_night]
 
    mjds = night_mjds

    # Get close (to of order body motion per day)
    times = astropy.time.Time(mjds, scale='utc', format='mjd', location=location)
    lsts = times.sidereal_time('apparent', longitude=location.lon)
    crds = astropy.coordinates.get_body(body, times, location=location)
    hour_angles = (lsts - crds.ra)
    event_hour_angles = event_direction * np.arccos(
        (np.sin(np.radians(alt)) - np.sin(crds.dec)*np.sin(location.lat))
        /(np.cos(crds.dec)*np.cos(location.lat)) )
    event_hour_angles = astropy.coordinates.Angle(event_hour_angles, unit=u.radian)
    ha_diff = (event_hour_angles - hour_angles).wrap_at(night_wrap*u.deg)
    mjds = mjds + ha_diff.radian*(0.9972696/(2*np.pi))
    
    # Refine using Newton's method
    for iter_idx in range(max_iter):
        times = astropy.time.Time(mjds, scale='utc', format='mjd', location=location)
        crds = astropy.coordinates.get_body(body, times, location=location)
        current_alt = crds.transform_to(astropy.coordinates.AltAz(obstime=times, location=location)).alt
        finished = np.max(np.abs(current_alt.deg - alt)) < tolerance
        if finished:
            break
            
        current_sinalt = np.sin(current_alt.rad)
        target_sinalt = np.sin(np.radians(alt))

        ha = times.sidereal_time('apparent') - crds.ra
        # Derivative of the standard formula for sin(alt) in terms of decl, latitude, and HA
        dsinalt_dlst = (-1*np.cos(crds.dec)*np.cos(location.lat)*np.sin(ha)).value
        dsinalt_dmjd = dsinalt_dlst * (2*np.pi/0.9972696)
        mjds = mjds - (current_sinalt - target_sinalt)/dsinalt_dmjd
    
    if np.max(np.abs(mjds - night_mjds)) > 1:
        warn("On some nights, found twilight more than a day away from the night mjd")
    
    if not finished:
        warn("twilight_times did not converge")
    
    return mjds

In [ ]:
schedule['evening'] = twilight_times(schedule.mjd, 'down')
schedule['morning'] = twilight_times(schedule.mjd, 'up')
schedule['midnight'] = 0.5*(schedule.morning+schedule.evening)
schedule['duration'] = 0.5*(schedule.morning-schedule.evening)

In [ ]:
schedule.describe().T

In [ ]:
start_mjd = min(schedule.mjd.min(), completed.mjd.min())-1
end_mjd = max(schedule.mjd.max(), completed.mjd.max())+2
mjds = np.arange(start_mjd, end_mjd, 1)
progress = pd.DataFrame({'mjd': mjds, 'exptime': 0, 'duration': 0})
progress.set_index('mjd', inplace=True)
progress['duration'] = schedule.groupby('mjd').duration.sum()
progress.fillna(0, inplace=True)
progress['cum_sched'] = progress.duration.cumsum()

completed['night_mjd'] = np.floor(completed.mjd - 0.9)
completed['obstime'] = completed.exptime + 30
progress['all_obstime'] = completed.groupby('night_mjd').agg({'obstime': 'sum'})
progress['good_obstime'] = completed.query('qc_teff>0.2').groupby('night_mjd').agg({'obstime': 'sum'})
progress.fillna(0, inplace=True)
progress['cum_all_obstime'] = progress.all_obstime.cumsum()/(24*60*60)
progress['cum_good_obstime'] = progress.good_obstime.cumsum()/(24*60*60)

progress.reset_index(inplace=True)
progress['date'] = MJD_EPOCH + progress.mjd.apply(lambda d: pd.Timedelta(d, unit='D'))

progress.describe().T

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1, 1, 1)
past = progress[progress['date'] < pd.Timestamp.now(tz=progress['date'][0].tz)]
past.plot('date', 'cum_good_obstime', drawstyle="steps-post", c='k', ax=ax, label='good observing time')
past.plot('date', 'cum_all_obstime', drawstyle="steps-post", c='orange', ax=ax, label='all observing time')
past.plot('date', 'cum_sched', c='r', drawstyle="steps-post", ax=ax, label='scheduled time')

## Overall depth map

Plot the total `qc_teff * exptime` by healpixel, giving a rough idea of our coverage. For the DELVE wide target of 3 by 90 second tilings with `qc_teff>0.3`, this would be `3*90*0.3=81` seconds.

In [ ]:
def teff_map(exposures, nside=32, **kwargs):
    exposures = exposures.copy()
    exposures['hpix'] = healpy.pixelfunc.ang2pix(nside, 
                                                 exposures['ra'], 
                                                 exposures['decl'],
                                                 lonlat=True)
    exposures['totteff'] = exposures['exptime']*exposures['qc_teff']
    depth_df = exposures.groupby('hpix').totteff.sum().reset_index().set_index('hpix', drop=False)
    npix = healpy.nside2npix(nside)
    depth = np.zeros(npix)
    depth[depth_df.hpix] = depth_df.totteff * DECAM_AREA / healpy.nside2pixarea(nside, degrees=True)
    healpy.azeqview(np.ma.masked_where(depth==0, depth),
                rot=(0, -90, 0), reso=18, lamb=True, **kwargs)
    healpy.visufunc.graticule()


In [ ]:
def multi_teff_map(completed, nside=32, bands=['u','g','r','i','z','Y','VR'], size=4, ncols=3, **kwargs):
    nbands = len(bands)
    nrows = int(np.ceil(nbands/ncols))
    fig, axes2d = plt.subplots(nrows, ncols, figsize=(ncols*size, nrows*size*1.2))
    axes = axes2d.flatten()
    for ax, band in zip(axes[:nbands], bands):
        plt.sca(ax)
        exposures = completed.query(f'qc_teff>0.3 and band=="{band}"').copy()
        teff_map(exposures, hold=True, title=f"Total t_eff, {band} band", **kwargs)
    
    for ax in axes[nbands:]:
        ax.set_visible(False)

In [ ]:
multi_teff_map(decam_completed, bands=['g','r','i','z','Y','VR'], min=0, max=90, cmap='viridis')